## Logging into Instagram:

In [17]:
import requests, re, json, csv
from urllib import parse
from bs4 import BeautifulSoup
from config import username, password

BASE_URL = 'https://www.instagram.com/accounts/login/'
LOGIN_URL = BASE_URL + 'ajax/'

session = requests.Session()
session.headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'}
session.headers.update({'referer': BASE_URL})    

req = session.get(BASE_URL)    
soup = BeautifulSoup(req.content, 'html.parser')    
body = soup.find('body')
pattern = re.compile('window._sharedData')
script = body.find("script", text=pattern)
script = script.contents[0].replace('window._sharedData = ', '')[:-1]
data = json.loads(script)
csrf = data['config'].get('csrf_token')
session.headers.update({'X-CSRFToken': csrf})

login_info = {
    'username': username,
    'password': password,
}
resp = session.post('https://www.instagram.com/accounts/login/ajax/', data=login_info, allow_redirects=True)
#login established


## Loading Account's Followers into a set:

In [18]:

GRAPHQL_API_URL = 'https://www.instagram.com/graphql/query/?'
user_info = json.loads(resp.content)
user_id = user_info['userId']

account_follower_ids = set()

#enumerate followers
control = {
    'query_hash':'c76146de99bb02f6415203be841dd25a', # magic value for Mark's following
    'variables': f'{{"id":"{user_id}","include_reel":true,"fetch_mutual":true,"first":50}}'
}
url_control = parse.urlencode(control)
#get first 50
graphql_resp = json.loads(session.get(GRAPHQL_API_URL + url_control).content)
has_next_page = graphql_resp['data']['user']['edge_followed_by']['page_info']['has_next_page']
user_nodes = graphql_resp['data']['user']['edge_followed_by']['edges']

open(r'followers.csv', 'w').close()
while True:
    for node in user_nodes:
        fields=[node['node']['username'],node['node']['full_name'],node['node']['id']]
        with open(r'followers.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow(fields)
        account_follower_ids.add(node['node']['id'])
    if not has_next_page:
        break
    end_cursor = graphql_resp['data']['user']['edge_followed_by']['page_info']['end_cursor']
    control = {
        'query_hash':'c76146de99bb02f6415203be841dd25a', # magic value for mark
        'variables': f'{{"id":"{user_id}","include_reel":true,"fetch_mutual":false,"first":50,"after":"{end_cursor}"}}'
    }
    url_control = parse.urlencode(control)
    #get next 50
    graphql_resp = json.loads(session.get(GRAPHQL_API_URL + url_control).content)
    has_next_page = graphql_resp['data']['user']['edge_followed_by']['page_info']['has_next_page']
    user_nodes = graphql_resp['data']['user']['edge_followed_by']['edges']
    
print(len(account_follower_ids))


2991


## Loading Users that the account is following into a set:

In [19]:
GRAPHQL_API_URL = 'https://www.instagram.com/graphql/query/?'
user_info = json.loads(resp.content)
user_id = user_info['userId']

account_following_ids = set()

#enumerate following
control = {
    'query_hash':'d04b0a864b4b54837c0d870b0e77e076', # magic value for Mark's following
    'variables': f'{{"id":"{user_id}","include_reel":true,"fetch_mutual":true,"first":50}}'
}
url_control = parse.urlencode(control)
#get first 50
graphql_resp = json.loads(session.get(GRAPHQL_API_URL + url_control).content)
has_next_page = graphql_resp['data']['user']['edge_follow']['page_info']['has_next_page']
user_nodes = graphql_resp['data']['user']['edge_follow']['edges']

open(r'following.csv', 'w').close()
while True:
    for node in user_nodes:
        fields=[node['node']['username'],node['node']['full_name'],node['node']['id']]
        with open(r'following.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow(fields)
        account_following_ids.add(node['node']['id'])
    if not has_next_page:
        break
    end_cursor = graphql_resp['data']['user']['edge_follow']['page_info']['end_cursor']
    control = {
        'query_hash':'d04b0a864b4b54837c0d870b0e77e076', # magic value for mark
        'variables': f'{{"id":"{user_id}","include_reel":true,"fetch_mutual":false,"first":50,"after":"{end_cursor}"}}'
    }
    url_control = parse.urlencode(control)
    #get next 50
    graphql_resp = json.loads(session.get(GRAPHQL_API_URL + url_control).content)
    has_next_page = graphql_resp['data']['user']['edge_follow']['page_info']['has_next_page']
    user_nodes = graphql_resp['data']['user']['edge_follow']['edges']
    
print(len(account_following_ids))

6065


## Obtaining which accounts you are following, who are not following you

In [20]:
following_unfollowers = account_following_ids - account_follower_ids
print(len(following_unfollowers))

following_data_dict = {}
with open('following.csv', newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        following_data_dict[row[2]] = row[0]

usernames_of_unfollowers = [following_data_dict[id] for id in list(following_unfollowers)]
print(len(usernames_of_unfollowers))


3514
3514


## Unfollowing them at a pace fast enough to not get banned

In [21]:
import time
import random 

list_unfollowers = list(following_unfollowers)

unfollow_url = 'https://www.instagram.com/web/friendships/{}/unfollow/'
USER_BASE_URL = 'https://www.instagram.com/{}/'
for uid in list_unfollowers:
    req = session.get(USER_BASE_URL.format(following_data_dict[uid]))    
    soup = BeautifulSoup(req.content, 'html.parser')    
    body = soup.find('body')
    pattern = re.compile('window._sharedData')
    script = body.find("script", text=pattern)
    script = script.contents[0].replace('window._sharedData = ', '')[:-1]
    data = json.loads(script)
    csrf = data['config'].get('csrf_token')
    session.headers.update({'X-CSRFToken': csrf})
    session.headers.update({'referer': USER_BASE_URL.format(following_data_dict[uid])})  
    print(following_data_dict[uid])
    resp = session.post(unfollow_url.format(uid))
    print(resp)
    while resp.status_code != 200:
        print('-----waiting on IG...-----')
        time.sleep(60*4 + random.randint(1, 10))
        resp = session.post(unfollow_url.format(uid))
    time.sleep(60*4 + random.randint(1, 10))

JSONDecodeError: Expecting value: line 2 column 1 (char 1)